# 部署活动和过滤器 <a class="anchor" id="top"></a>

在这个notebook中，您将在Amazon Personalize中部署活动并与之互动。

1. [简介](#intro)
1. [创建活动](#create)
1. [与活动互动](#interact)
1. [批量推荐](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[回到顶部](#top)

此时，您应该具有多个解决方案，并且每个解决方案至少具有一个解决方案版本。一旦创建了解决方案版本，就可以从它们那里获得推荐，并对他们的整体行为有所了解。

该笔记本首先将先前笔记本中的每个解决方案版本部署到单独的活动中。 一旦它们处于活动状态，就有资源可供查询推荐，并且可以使用辅助函数将输出摘要为更易于理解的内容。


首先，我们需要再次导入库，从以前的notebook中加载值，并加载SDK。

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [ ]:
%store -r

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# 建立一个到Personalize'的事件流
personalize_events = boto3.client(service_name='personalize-events')

## 创建活动 <a class="anchor" id="create"></a>
[回到顶部](#top)

活动是一个托管的解决方案版本；一个您可以查询建议的终端节点。在部署一个活动时，您可以设置最低每秒吞吐量（TPS）值。 与亚马逊云科技中的许多服务一样，该服务将根据需求自动扩展，但是如果延迟至关重要，则您可能需要提前预置以满足更大的需求。 对于此POC和演示，所有最小吞吐量阈值均设置为1。更多信息请查看[价格页面](https://aws.amazon.com/personalize/pricing/)

让我们开始部署这个活动。

### 用户个性化

为您的用户个性化解决方案版本部署活动。 部署一个活动大约需要10分钟的时间。 通常，我们将使用while循环进行轮询，直到任务完成为止。 不过这个while任务将阻止notebook其他单元格代码的执行，此处我们的目标是创建多个活动。因此，我们将在notbook中为所有活动设置while循环。 此外，您还将找到有关在亚马逊云科技控制台中查看进度的说明。

In [ ]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

### SIMS

为您的SIMS解决方案版本部署活动。 部署一个活动大约需要10分钟的时间。 通常，我们将使用while循环进行轮询，直到任务完成为止。 不过这个while任务将阻止notebook其他单元格代码的执行，此处我们的目标是创建多个活动。因此，我们将在notbook中为所有活动设置while循环。 此外，您还将找到有关在亚马逊云科技控制台中查看进度的说明。

In [ ]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

### 个性化排名

为您的个性化排名解决方案版本部署活动。 部署一个活动大约需要10分钟的时间。 通常，我们将使用while循环进行轮询，直到任务完成为止。 不过这个while任务将阻止notebook其他单元格代码的执行，此处我们的目标是创建多个活动。因此，我们将在notbook中为所有活动设置while循环。 此外，您还将找到有关在亚马逊云科技控制台中查看进度的说明。

In [ ]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

### 查看活动创建状态

在我们前面所提到的，如何在控制台中查看状态更新:

* 在另一个浏览器选项卡中，打开亚马逊云科技控制台。
* 切换到该标签并在顶部搜索`Personalize`服务，然后转到该服务页面。
* 点击 `查看数据集组`.
* 点击数据集组的名称，一般来说是名称中带有“POC”字样的那个。
* 点击 `活动`.
* 现在您将看到上面创建的所有活动的列表，包括带有活动状态的列。 一旦它的状态为`Active`，你的活动就已经准备好接受查询。

或者，只需运行下面的单元格中的代码即可跟踪活动的创建状态。

In [ ]:
in_progress_campaigns = [
    userpersonalization_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

## 创建过滤器 <a class="anchor" id="interact"></a>
[回到顶部](#top)

现在，所有活动均已部署并处于活动状态，我们可以创建过滤器。 可以为“项目”和“事件”创建过滤器。 视频点播中的过滤器的一些常见用例是：

基于项目元数据的分类过滤器：通常，您的项目元数据会包含有关您的标题的信息，例如流派，关键字，年份，年代等。对此进行过滤可以在该数据中提供建议，例如动作片。

事件：您可能希望过滤掉某些事件并根据这些事件提供结果，例如将标题从“要观看的建议”建议移到“再次观看”建议。

让我们看一下项目元数据和用户交互，这样我们就可以知道可以创建哪种类型的过滤器。

In [ ]:
# 通过读取正确的源CSV为物品创建dataframe
items_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)
#interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# 查看一些示例数据
items_df.head(10)
#interactions_df.head(10)

现在，我们要做的是确定要筛选的类型，为此，我们需要所有类型的列表。 首先，我们将获取GENRE列的所有唯一值，然后在`|`上拆分字符串（如果存在的话），然后将每一个添加到一个长列表中，该列表将转换为集合以提高效率。 然后将该集合放入一个列表中，以便可以对其进行迭代，然后我们可以使用创建过滤器的API。

In [ ]:
unique_genre_field_values = items_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [ ]:
genres_to_filter

现在，我们有了数据集中存在的所有类型。 目前，Personalize的一个软限制是总共10个过滤器，如果我们拥有更多类型，我们将随机选择7个，以便稍后为2个基于交互的过滤器留出空间，并为基于年份的推荐留出额外的过滤器。

In [ ]:
genres_to_filter = random.sample(genres_to_filter, 7)
genres_to_filter

现在为元数据类型过滤器创建一个列表，然后使用下面单元格中的代码创建实际的过滤器。 请注意，这将需要几分钟才能完成。

In [ ]:
# 创建过滤器列表:
meta_filter_arns = []

In [ ]:
# 遍历类型
for genre in genres_to_filter:
    # 首先创建一个过滤器
    try:
        createfilter_response = personalize.create_filter(
            name=genre,
            datasetGroupArn=dataset_group_arn,
            filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
        )
        # 将ARN添加到列表
        meta_filter_arns.append(createfilter_response['filterArn'])
        print("Creating: " + createfilter_response['filterArn'])
    
    # 如果失败，稍等一会儿
    except ClientError as error:
        # 这里我们只关心是否出现的是出现限流的问题
        if error.response['Error']['Code'] != 'LimitExceededException':
            print(error)
        else:    
            time.sleep(120)
            createfilter_response = personalize.create_filter(
                name=genre,
                datasetGroupArn=dataset_group_arn,
                filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
            )
            # 将ARN添加到列表
            meta_filter_arns.append(createfilter_response['filterArn'])
            print("Creating: " + createfilter_response['filterArn'])

让我们还为“已观看”和“未观看”的内容创建2个事件过滤器

In [ ]:
# 通过读取正确的源CSV创建用于交互的dataframe
interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# 查看一些示例数据
interactions_df.head(10)

让我们还为“已观看”和“未观看”的内容创建2个事件过滤器

In [ ]:
createwatchedfilter_response = personalize.create_filter(name='watched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )

createunwatchedfilter_response = personalize.create_filter(name='unwatched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='EXCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )


最后，由于我们在项目元数据中提供了年份数据，因此创建一个十年过滤器以仅推荐给定年代中发行的电影，为此workshop，我们将选择1970年代的影片。

In [ ]:
createdecadefilter_response = personalize.create_filter(name='1970s',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Items.YEAR >= 1970 AND Items.YEAR < 1980'
    )

在完成之前，我们将希望将这些过滤器也添加到列表中，以便以后使用。

In [ ]:
interaction_filter_arns = [createwatchedfilter_response['filterArn'], createunwatchedfilter_response['filterArn']]

In [ ]:
decade_filter_arns = [createdecadefilter_response['filterArn']]

In [ ]:
%store sims_campaign_arn
%store userpersonalization_campaign_arn
%store rerank_campaign_arn
%store meta_filter_arns
%store interaction_filter_arns
%store decade_filter_arns


您已经准备好进入最后一个探索性笔记本：'05_Interacting_with_Campaigns_and_Filters.ipynb'。 从浏览器中打开它，您就可以开始与活动进行互动并获取推荐了！